**RMS_bins.ipynb**

This notebook generates the look-up table "CINDI_RMS_LUT.csv".
The look-up table contains the RMS thresholds, defined by the 95 percentiles for each Pandora ID, dataproduct, reference type and elevation angle. The RMS thresholds are later used to filter the data in the defined bins.

In this notebook, the unfiltered CINDI3 data are binned in 6 (or 10) bins of SZA for all routines during elevation scan phase (or during twilight phase (routines 'IW')). The 95 percentile RMS is calculated for each bin and stored in a pandas.DataFrame LUT in the variable "rms" together with the respective bin edges "sza". The bin edges are split in two lists of the lower edges "sza_left" and the upper edges "sza_right" for better readability in the produced .csv file.

How to generate / use this script:
1) Run "main_makeCompData.py" for all Pandora IDs, dates, dataproducts and reference types. SZA binning should be disabled (last row in "processCompDataInput.txt": *SZA binning for RMS filtering (...) -> **NO***). Make to enable the conversion to CINDI3 blind comparison format. Move the files to folders named as the dates ("YYYYMMDD").
2) Specifiy the path to the folder containing all date folders in the next cell, and run all following cells of this notebook. Store the variable LUT in ".csv" format (last cell).
3) If it's not in the correct path, move "CINDI_RMS_LUT.csv" to the root path of "main_makeCompData.py". Run "main_makeCompData.py this time with SZA binning (last row in "processCompDataInput.txt" set to *SZA binning for RMS filtering (...) -> **FROMFILE***).

In [ ]:
import os
import xarray as xa
import numpy as np
from copy import copy
import pandas as pd
import itertools

# navigate to folder that contains subfolders of dates '20240527', '20240528', ..., '20240621'
# subfolders of dates contain CINDI-3 submission files
path='C:/Users/Stefanie Morhenn/PycharmProjects/Blick/src/CINDI3/v1.6 (no RMS filtering)'

In [ ]:
os.chdir(path)

dates = os.listdir()
data = xa.Dataset(
    data_vars=dict(
        RMS=(['PAN_ID','DATE','REF','DATAPRODUCT','VEA','TIME_INDEX'],np.zeros((3,len(dates),3,12,32,1000))*np.nan),
        SZA=(['PAN_ID','DATE','REF','DATAPRODUCT','VEA','TIME_INDEX'],np.zeros((3,len(dates),3,12,32,1000))*np.nan),
        RTN=(['PAN_ID','DATE','REF','DATAPRODUCT','VEA','TIME_INDEX'],np.empty((3,len(dates),3,12,32,1000),dtype='<U2')),
        ),
    coords=dict(PAN_ID=('PAN_ID',np.array(['34','35','36'])), 
                DATE=('DATE', np.array(dates)), 
                REF=('REF',np.array(['DAILYREF', 'SEQREF','FIXREF'])), 
                DATAPRODUCT=('DATAPRODUCT',np.array(['O4VIS','O4UV','O3VIS','O3UV','NO2VIS-SMALL','NO2VIS','NO2UV','HONO','HCHO-WIDE','HCHO','CHOCHO','BRO'])), 
                VEA=('VEA', np.array([-5.,-4.,-3.,-2.,-1.8,-1.6,-1.2,-1.1,-1.,-0.8,-0.6,-0.4,-0.2,0.,0.2,0.4,0.6,0.8,1.,1.2,1.4,1.6,1.8,2.,3.,4.,5.,6.,8.,15.,30.,90.])), 
                TIME=('TIME_INDEX', np.arange((1000))),
    )
)

for date in dates:
    os.chdir(date)
    for file in os.listdir():
        p_id = file.split('_')[2]
        ref = file.split('_')[4]
        dataproduct = file.split('_')[3]
        with open(file,'r') as f:
            i_rms_found=False
            iw_finished=False
            sza = []
            vea = []
            rms = []
            rtn = []
            lines = f.readlines()
            for iline, line in enumerate(lines):
                if '% ' in line[:2]:
                    # index of rms column in file
                    if 'Fit RMS' in line[2:] and 'Col' in line[2:]:
                        # index is written number behind '% Col ' - 1
                        i_rms = int(line.split('Col ')[-1].split(':')[0]) - 1
                else:
                    linelist = line.split(' ')
                    # extract time for distinction between twilight and non-twilight measurements
                    time = float(linelist[1])
                    hh = int(time)
                    mm = int((time-hh)*60)
                    ss = (((time-hh)*60)-mm)*60
                    sza.append(float(linelist[3]))
                    vea.append(float(linelist[5]))
                    rms.append(float(linelist[i_rms]))
                    # set flag prev_elev_scan_complete to False if a new elevation scan begins
                    if vea[-1] == -2 and iw_finished:
                        prev_elev_scan_complete = False
                    if vea[-1] == 90 and iw_finished:
                        prev_elev_scan_complete = True
                    # make distinction between twilight measurements ('IW') and non-twilight measurements ('IX'), based on timestamp and vea in CINDI-3 file
                    # first: early twilight measurements
                    if vea[-1] != 90 and not iw_finished:
                        iw_finished=True
                        i_first_elev_scan = copy(iline)
                    # then: late twilight measurements
                    if time >= 19+1/6 and iw_finished:
                        if prev_elev_scan_complete:
                            iw_finished=False
                        else:
                            print(p_id)
                            print(ref)
                            print(dataproduct)
                            print(date)
                            print('Please check if VEA=%s, timestamp=%s is associated to last measurement of elevation scan or to IW routine. We will assume it belongs to IW.' % (vea[-1],time))
                            iw_finished=False
                    rtn.append('IX' if iw_finished else 'IW')
            for i_angle, angle in enumerate(vea):
                data.RMS.sel(PAN_ID=p_id,DATE=date,REF=ref,DATAPRODUCT=dataproduct,VEA=angle).isel(TIME_INDEX=slice(0,len(np.array(rms))))[:] = np.array(rms)
                data.SZA.sel(PAN_ID=p_id,DATE=date,REF=ref,DATAPRODUCT=dataproduct,VEA=angle).isel(TIME_INDEX=slice(0,len(np.array(sza))))[:] = np.array(sza)
                data.RTN.sel(PAN_ID=p_id,DATE=date,REF=ref,DATAPRODUCT=dataproduct,VEA=angle).isel(TIME_INDEX=slice(0,len(np.array(rtn))))[:] = np.array(rtn)

    os.chdir('..')

In [ ]:
# Binning for each instrument, dataproduct, reference, VEA
LUT=pd.DataFrame(list(itertools.product(
    data.PAN_ID.data, data.DATAPRODUCT.data, data.REF.data, data.VEA.data, ['IW','IX']
)),
columns=['pan_id','dataproduct','ref','vea','rtn'])
LUT['rms'] = [[] for _ in range(len(LUT))]
LUT['sza'] = [[] for _ in range(len(LUT))]
LUT_rows = []
for index, row in LUT.iterrows():
    data_sub = data.sel(PAN_ID=row.pan_id,DATAPRODUCT=row.dataproduct,REF=row.ref)
    data_sub = data_sub.where((data_sub.VEA==row.vea) & (data_sub.RTN==row.rtn),drop=True)

    if data_sub.RTN.shape == (0,0,0):
        continue
    if row.rtn == 'IW':
        stepsize=10
    else:
        stepsize=6
    bin_edges = np.linspace(np.nanmax(data_sub.SZA)+1e-3,np.nanmin(data_sub.SZA),stepsize+1)
    data_sub_binned = data_sub.groupby_bins('SZA',bins=bin_edges[::-1],right=False)
    for label, group in data_sub_binned:
        threshold=group['RMS'].quantile(0.95)
        LUT.at[index,'rms'].append(threshold.data.item())
        LUT.at[index,'sza'].append(label)

LUT['pan_id']=LUT['pan_id'].replace('34','118')
LUT['pan_id']=LUT['pan_id'].replace('35','81')
LUT['pan_id']=LUT['pan_id'].replace('36','83')
LUT['ref']=LUT['ref'].replace('DAILYREF','Ref')
LUT['ref']=LUT['ref'].replace('FIXREF','RefFix')
LUT['ref']=LUT['ref'].replace('SEQREF','MeasLow')

list_sza_left = []
list_sza_right = []
for index in LUT.index:
    sza_left=[i.left for i in LUT.loc[index]['sza']]
    sza_right=[i.right for i in LUT.loc[index]['sza']]
    list_sza_left.append(sza_left)
    list_sza_right.append(sza_right)
LUT['sza_left'] = list_sza_left
LUT['sza_right'] = list_sza_right

In [ ]:
LUT.drop(columns=['sza']).to_csv(os.path.join(path,'CINDI_RMS_LUT.csv'),index=False)